# Theme

**Table of contents:**
 - [Problem formulation](#Problem-formulation)
 - [Creating sample paths](#Creating-sample-paths)
 - [Writing text files](#Writing-text-files)
 - [Writing binary files](#Writing-binary-files)
 - [Writing HDF5 files](#Writing-HDF5-files)
 - [Visualization](#Visualization)
 - [A Python version](#a-python-version)
 - [Discussion](#Discussion)

## Problem formulation

Think of a device that over a certain epoch produces time series of measurements. The device could be a physical sensor or a numerical device such as a simulation, or it could be the price of a financial instrument such as a stock. Let's assume we want to capture such time series over many epochs and that each epoch has a fixed number of time steps. A visualization of five such series is shown in the figure (**Source:** __[Wikimedia](https://en.wikipedia.org/wiki/File:Ornstein-Uhlenbeck-5traces.svg)__) below.

<img src="img/Ornstein-Uhlenbeck-5traces.png" title="Five sampled traces of an Ornstein-Uhlenbeck process." />

Assuming that an epoch of length 10 was sampled over 1,000 equidistant time steps, we could conveniently store the underlying data (values of `X`) in a `5 x 1000` two-dimensional array of floating-point values, with the series number and the time step as the row and column index, respectively.

For our own benefit and the researchers' with whom we might want to share our observations, we should also record relevant information about the context in which the data was collected. This might be for example the unit of the quantity `X`, the length of an individual time step, the date, time, and location when the recording was made (if that were relvant), and any additional information about the calibration of the generating mechanism. To be specific, in this tutorial, we will use numerical samples of a well-known stochastic process, the so-called __[Ornstein–Uhlenbeck process](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process)__, as our data source. The sample traces or path samples shown in the figure were actually generated from a numerical experiment of such a process.

<div class="alert alert-block alert-success">
<b>Up to you:</b> To follow this tutorial, it is <i>not</i> necessary to understand what a stochastic process is or how to simulate one. It is OK to think about it as a sophisticated pseudo-random number generator, which merely serves as our source of series data.
</div>

Our problem formulation is thus: Store the floating-point values of `path_count` series with `step_count` values each in a two-dimensional array of `path_count` rows and `step_count` columns. In addition, store the following floating-point calibrations, which are the same for all series
- the time step `dt`
- the long-term process mean `mu`
- the reversion rate to the mean `theta`
- the volatility of the process `sigma`.

Before thinking about how to store this data, we take a quick look at how the series data is generated. (It won't come as a shock that pseudo-random numbers play a big part!) With that out of the way, we look at storing the data in plain text files, as unformatted byte streams, and in HDF5 files. We conclude with a discussion of the advantages and challenges of the three approaches.

## Creating sample paths

We create `path_count` sample paths with `step_count` timesteps with the given parameters, and store them in a C++ `vector`.

In [ ]:
%%writefile src/ou_sampler.hpp
#ifndef OU_SAMPLER_HPP
#define OU_SAMPLER_HPP

#include <vector>

// Creates `path_count` sample paths of length `step_count` with parameters
// `dt`, `theta`, `mu`, and `sigma`
extern void ou_sampler
(
    std::vector<double>& ou_process,
    const size_t&        path_count,
    const size_t&        step_count,
    const double&        dt,
    const double&        theta,
    const double&        mu,
    const double&        sigma
);

#endif

We use C++ __[`random::normal_distribution`](https://en.cppreference.com/w/cpp/numeric/random/normal_distribution)__ to generate the random increments with mean 0 and standard deviation $\sqrt{dt}$ for the sample paths.

In [ ]:
%%writefile src/ou_sampler.cpp

#include <random>

using namespace std;

void ou_sampler
(
    vector<double>& ou_process,
    const size_t&   path_count,
    const size_t&   step_count,
    const double&   dt,
    const double&   theta,
    const double&   mu,
    const double&   sigma
)
{
    // Store sample paths in one contiguous buffer
    ou_process.clear();
    ou_process.resize(path_count * step_count);

    random_device rd;
    mt19937 generator(rd());
    normal_distribution<double> dist(0.0, sqrt(dt));

    for (size_t i = 0; i < path_count; ++i)
    {
        ou_process[i * step_count] = 0; // sample paths start at x = 0
        for (size_t j = 1; j < step_count; ++j)
        {
            auto dW = dist(generator);
            auto pos = i * step_count + j;
            ou_process[pos] = ou_process[pos - 1] + theta * (mu - ou_process[pos - 1]) * dt + sigma * dW;
        }
    }
}

## Writing text files

The easiest way to store the sample paths would be a text file. We include a header in which we document the parameters that were uses in the generation. This is adequate as long as the number of sample paths and timesteps is relatively small. For large numbers, this is cumbersome and, eventually, unmanagable.

In [ ]:
%%writefile src/ou_text.cpp
#include "ou_sampler.hpp"

#include <fstream>
#include <iostream>
#include <vector>

using namespace std;

int main()
{
    const size_t path_count = 100, step_count = 1000;
    const double dt = 0.01, theta = 1.0, mu = 0.0, sigma = 0.1;

    cout << "Running with parameters:"
         << " paths=" << path_count << " steps=" << step_count
         << " dt=" << dt << " theta=" << theta << " mu=" << mu << " sigma=" << sigma << endl;

    vector<double> ou_process;
    ou_sampler(ou_process, path_count, step_count, dt, theta, mu, sigma);
    
    // Write the sample paths to a text file
    ofstream file("ou_process.txt");
    
    file << "# paths steps dt theta mu sigma" << endl;
    file << path_count << " " << step_count << " " << dt << " " << theta << " " << mu << " " << sigma << endl;
    file << "# data" << endl;
    
    for (size_t i = 0; i < path_count; ++i)
        {
            for (size_t j = 0; j < step_count; ++j)
            {
                auto pos = i * step_count + j;
                file << ou_process[pos] << " ";
            }
            file << endl;
        }

    file.close();

    return 0;
}

In [ ]:
%%bash
g++ -std=c++17 -Wall -pedantic -I./include ./src/ou_text.cpp ./src/ou_sampler.cpp -o ./build/ou_text
./build/ou_text
ls -iks ou_process.txt

## Writing binary files

To reduce the file size, instead of using text, we can store the sample paths as an unformatted binary stream. We would need to maintain separate documentation to tell consumers of the data how to parse and interpret this byte stream, which is a minor inconvenience. A major headache is that unformatted binary streams are platform (processor) dependent, which opens the door for misinterpretation when the data is copied to an incompatible platform and not adjusted for this change.

In [ ]:
%%writefile src/ou_binary.cpp
#include "ou_sampler.hpp"

#include <fstream>
#include <iostream>
#include <vector>

using namespace std;

int main()
{
    const size_t path_count = 100, step_count = 1000;
    const double dt = 0.01, theta = 1.0, mu = 0.0, sigma = 0.1;

    cout << "Running with parameters:"
         << " paths=" << path_count << " steps=" << step_count
         << " dt=" << dt << " theta=" << theta << " mu=" << mu << " sigma=" << sigma << endl;

    vector<double> ou_process;
    ou_sampler(ou_process, path_count, step_count, dt, theta, mu, sigma);
    
    // Write the sample paths to an unformatted binary file

    ofstream file("ou_process.bin", ios::out | ios::binary);
    file.write((char *)&path_count, sizeof(path_count));
    file.write((char *)&step_count, sizeof(step_count));
    file.write((char *)&dt, sizeof(dt));
    file.write((char *)&theta, sizeof(theta));
    file.write((char *)&mu, sizeof(mu));
    file.write((char *)&sigma, sizeof(sigma));
    file.write((char *)ou_process.data(), sizeof(double) * ou_process.size());
    file.close();

    return 0;
}

In [ ]:
%%bash
g++ -std=c++17 -Wall -pedantic -I./include ./src/ou_binary.cpp ./src/ou_sampler.cpp -o ./build/ou_binary
./build/ou_binary
ls -iks ou_process.bin

## Writing HDF5 files

Unlike plain text and unformatted binary streams, in HDF5 files, users can arrange many multidimensional arrays in hierarchies, which resemble folder/file structures in file systems (folder = HDF5 group, file = HDF5 dataset). These hierarchies can be navigated like file systems, and entities can be referenced via HDF5 path names (`/foo/bar/qux`). Furthermore, these entities can be decorated with key/value-like metadata, so-called HDF5 attributes.

Below, we show a simple example of a function that decorates an HDF5 object, identified by a handle `loc` and a name `rel` relative to `loc`, with a documentation string `val` under the key `key`.

In [ ]:
%%writefile src/docstring.hpp
#ifndef DOCSTRING_HPP
#define DOCSTRING_HPP

#include "hdf5.h"
#include <string>

// Writes a scalar HDF5 string attribute
extern void add_docstring
(
    hid_t&             loc,
    const std::string& name,
    const std::string& key,
    const std::string& value
);

#endif

In [ ]:
%%writefile src/docstring.cpp
#include "docstring.hpp"

using namespace std;

void add_docstring(hid_t& loc, const string& name, const string& key, const string& value)
{
    hid_t scalar = H5Screate(H5S_SCALAR), strtype = H5Tcopy(H5T_C_S1);
    H5Tset_size(strtype, value.size());
    H5Tset_strpad(strtype, H5T_STR_NULLTERM);
    hid_t attr = H5Acreate_by_name(loc, (name.size() >= 1) ? name.c_str() : ".", key.c_str(), strtype, scalar,
        H5P_DEFAULT, H5P_DEFAULT, H5P_DEFAULT);
    H5Awrite(attr, strtype, value.c_str());
    H5Aclose(attr);
    H5Tclose(strtype);
    H5Sclose(scalar);
}

We store the 2D array of sample paths in an HDF5 dataset named `/dataset`. In the HDF5 framework, creating (`H5Dcreate` on line 34) and writing (`H5Dwrite` on line 35) to a dataset are separate steps.

Along the way, we add a few documentation strings (lines 29, 41-44). Finally, we decorate the dataset with four attributes: `dt`, θ, μ, and σ (lines 55 to 58).

Notice the surrounding boilerplate of `H5*open,close` calls? Remember Powell's rule?

In [ ]:
%%writefile src/ou_hdf5.cpp
#include "docstring.hpp"
#include "ou_sampler.hpp"

#include "hdf5.h"
#include <iostream>
#include <vector>

using namespace std;

int main()
{
    const size_t path_count = 100, step_count = 1000;
    const double dt = 0.01, theta = 1.0, mu = 0.0, sigma = 0.1;

    cout << "Running with parameters:"
         << " paths=" << path_count << " steps=" << step_count
         << " dt=" << dt << " theta=" << theta << " mu=" << mu << " sigma=" << sigma << endl;

    vector<double> ou_process;
    ou_sampler(ou_process, path_count, step_count, dt, theta, mu, sigma);
    
    //
    // Write the sample paths to an HDF5 file using the HDF5 C-API!
    //

    auto file = H5Fcreate("ou_process.h5", H5F_ACC_TRUNC, H5P_DEFAULT, H5P_DEFAULT);

    add_docstring(file, ".", "source", "https://github.com/HDFGroup/hdf5-tutorial");

    { // create & write the dataset
        hsize_t dimsf[] = {(hsize_t)path_count, (hsize_t)step_count};
        auto space = H5Screate_simple(2, dimsf, NULL);
        auto dataset = H5Dcreate(file, "/dataset", H5T_NATIVE_DOUBLE, space, H5P_DEFAULT, H5P_DEFAULT, H5P_DEFAULT);
        H5Dwrite(dataset, H5T_NATIVE_DOUBLE, H5S_ALL, H5S_ALL, H5P_DEFAULT, ou_process.data());
        H5Dclose(dataset);
        H5Sclose(space);
    }

    { // make the file self-describing by adding a few attributes to `dataset`
        add_docstring(file, "dataset", "comment", "This dataset contains sample paths of an Ornstein-Uhlenbeck process.");
        add_docstring(file, "dataset", "Wikipedia", "https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process");
        add_docstring(file, "dataset", "rows", "path");
        add_docstring(file, "dataset", "columns", "time");
        
        auto scalar = H5Screate(H5S_SCALAR);
        auto acpl = H5Pcreate(H5P_ATTRIBUTE_CREATE);
        H5Pset_char_encoding(acpl, H5T_CSET_UTF8);
        
        auto set_attribute = [&](const string& name, const double& value) {
            auto attr = H5Acreate_by_name(file, "dataset", name.c_str(), H5T_NATIVE_DOUBLE, scalar, acpl, H5P_DEFAULT, H5P_DEFAULT);
            H5Awrite(attr, H5T_NATIVE_DOUBLE, &value);
            H5Aclose(attr);
        };
        set_attribute("dt", dt);
        set_attribute("θ", theta);
        set_attribute("μ", mu);
        set_attribute("σ", sigma);

        H5Pclose(acpl);
        H5Sclose(scalar);
    }

    H5Fclose(file);

    return 0;
}

In addition to the sampler, let's tell the compiler where to find the HDF5 headers and library!

In [ ]:
%%bash
g++ -std=c++17 -Wall -pedantic -I/usr/include/hdf5/serial -L/usr/lib/x86_64-linux-gnu -I./include  ./src/ou_hdf5.cpp ./src/ou_sampler.cpp ./src/docstring.cpp -o ./build/ou_hdf5 -lhdf5_serial
./build/ou_hdf5
ls -iks ou_process.h5

## Visualization

The easiest way to visualize the content of the HDF5 file `ou_process.h5` is to select it in the Explorer. This will launch the __[H5Web VS Code plugin](https://marketplace.visualstudio.com/items?itemName=h5web.vscode-h5web)__, which is sufficient for quick inspection and simple plot types.

<img src="img/H5Web.png" alt="H5Web" title="H5Web VSCode plugin." />

A more powerful option is __[Matplotlib](https://matplotlib.org/)__, and a plot of the fourty third sample path can be obtained as shown below.

In [ ]:
%matplotlib inline
import h5py
import numpy as np

f = h5py.File("ou_process.h5")
dset = f["dataset"]

In [ ]:
arr = dset[42,:]
print(f"min: {arr.min():.2f}, max: {arr.max():.2f}, mean: {arr.mean():.2f}")

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('_mpl-gallery')
fig, ax = plt.subplots()
ax.plot(np.arange(0,len(arr)), arr, linewidth=2.0)
plt.show()

In [ ]:
f.close()

## A Python version

A Python version of our C++ program can be easily obtained by running the C++ code through an AI assistant such as __[ChatGPT](https://chat.openai.com/)__ or __[GitHub Copilot](https://github.com/features/copilot/)__. The resulting Python code is shown below.

In [ ]:
import numpy as np
import h5py

def ou_sampler(path_count, step_count, dt, theta, mu, sigma):
    '''
    Generates sample paths for an Ornstein-Uhlenbeck process.
    '''
    ou_process = np.zeros((path_count, step_count))
    for i in range(path_count):
        for j in range(1, step_count):
            dW = np.random.normal(0, np.sqrt(dt))
            ou_process[i, j] = ou_process[i, j-1] + theta * (mu - ou_process[i, j-1]) * dt + sigma * dW
    return ou_process

def main():
    # Parameters
    path_count = 100
    step_count = 1000
    dt = 0.01
    theta = 1.0
    mu = 0.0
    sigma = 0.1

    print("Running with parameters:", "paths=", path_count, "steps=", step_count, "dt=", dt, "theta=", theta, "mu=", mu, "sigma=", sigma)

    # Generate OU process sample paths
    ou_process = ou_sampler(path_count, step_count, dt, theta, mu, sigma)

    # Write sample paths to an HDF5 file
    with h5py.File('ou_h5py.h5', 'w') as file:
        file.attrs['source'] = 'https://github.com/HDFGroup/hdf5-tutorial'

        # Create & write the dataset
        dataset = file.create_dataset('dataset', data=ou_process)

        # Add documentation to the dataset
        file['dataset'].attrs['comment'] = 'This dataset contains sample paths of an Ornstein-Uhlenbeck process.'
        file['dataset'].attrs['Wikipedia'] = 'https://en.wikipedia.org/wiki/Ornstein–Uhlenbeck_process'
        file['dataset'].attrs['rows'] = 'path'
        file['dataset'].attrs['columns'] = 'time'
        
        # Set attributes
        file['dataset'].attrs['dt'] = dt
        file['dataset'].attrs['θ'] = theta
        file['dataset'].attrs['μ'] = mu
        file['dataset'].attrs['σ'] = sigma

if __name__ == "__main__":
    main()

The code is not very Pythonic, but it works. Most importantly, it doesn't need a ton of boilerplate code. To be clear, the boilerplate in the C/C++ version is **not** a language problem, but the price to pay for a C-API that is designed to give users maximal control over the details of the data layout and I/O characteristics.

## Discussion

In this part of the tutorial, we demonstrated how to store a set of sample paths plus metadata in plain text, as an unformatted binary stream, and in an HDF5 file. Advantages and challenges are relative to circumstances, and pointless to belabor in the abstract. If circumstances change, it might be time for a change of approach. Here are a few considerations:

The simplicity of plain text is hard to beat for small data sets. No special tools or libraries are needed to handle text. (There are, however, __[challenges](https://youtu.be/_mZBa3sqTrI?si=sKQKvFgserRq2uHf)__ with "plain text" the moment you go beyond US-ASCII.) Performance in space (file size) and time (marshalling overhead) deteriorates with data size. Common mitigations include compression and parallel processing, but the original simplicity is gone.

Unformatted binary streams are fine as long as platforms and layouts are stable. The moment multiple platforms with different byte orders enter the mix, things get tricky. Change in the metadata or data is the biggest enemy. Not only is (separate) documentation in need of updates, but costly code changes to support legacy and new data might be necessary.

The HDF5 framework takes care of all of the issues mentioned so far at the expense of a dependency on the HDF5 library and file format or HSDS. Common considerations before deciding on making your product dependent on the HDF5 framework include:

1. **Licensing and Costs:** The HDF5 framework is open source and released under a __[permissive license](https://www.hdfgroup.org/licenses/)__, which allows for free use, modification, and redistribution of the software, both in source and binary forms, with minimal restrictions. As open-source software, the HDF5 framework is generally free to use. There are no costs associated with downloading and using the HDF5 library or HSDS in your projects. However, there might be costs associated with specific third-party tools or commercial applications that use HDF5 as part of their software.
2. **Compatibility and Integration:** The HDF5 framework is designed to be highly portable and runs on a wide range of hardware and operating system platforms, including Linux, macOS, and Windows. HDF5 is supported in various programming languages, including C, C++, Fortran, Java, Python, and many more through official and third-party bindings. The HDF5 data model is unique, and frequently copied in parts. If you are integrating HDF5 into an existing system, consider how HDF5's primitives (datasets, groups, attributes, etc.) map to your application’s data model.
3. **Support and Documentation:** The support and documentation for HDF5 are robust, catering to a wide range of users from beginners to advanced. The HDF Group provides extensive official documentation. This includes detailed guides on installation, API references, user guides for various programming languages, and explanations of HDF5 concepts. The documentation is regularly updated to reflect the latest developments. The HDF5 framework has a large and active user community. Community support is available through mailing lists, forums, and Q&A sites where users can ask questions and share knowledge. The HDF Forum is particularly active and a good resource for seeking help and advice. There are numerous tutorials, examples, and use cases available, both within the official documentation and from third-party sources. These resources are valuable for both new and experienced users in understanding how to effectively use HDF5 for different applications. The HDF Group offers __[commercial support](https://www.hdfgroup.org/solutions/priority-support/)__ and __[consulting services](https://www.hdfgroup.org/solutions/consulting/)__. This includes help with installation, optimization, data modeling, and other technical issues.
4. **Reliability and Performance:** The HDF5 framework is known for its reliability and performance, particularly in environments that require handling large and complex datasets. However, like any software, its performance and reliability can depend on various factors including usage patterns, system architecture, and specific application requirements. As a mature and widely-used technology, HDF5 is generally considered stable. It is used in critical applications across different domains and industries, which speaks to its reliability. For applications requiring parallel processing, HDF5 can be used with MPI (Message Passing Interface) to enable efficient parallel I/O operations. HDF5 allows users to tailor data storage layouts to match specific access patterns, which can greatly optimize read/write performance. HDF5 supports various data compression techniques, which can reduce file size and I/O overhead, especially beneficial for network transfers and storage efficiency.
5. **Updates and Longevity:** The HDF Group regularly releases updates. These updates include new features, performance improvements, bug fixes, and security patches. The HDF Group has a strong commitment to backward and forward compatibility. This means that newer versions of the HDF5 framework maintain compatibility with older versions, which is critical for long-term data accessibility. At the same time, older versions of the framework can access data stored using a newer version as long as no features available only in later versions were used in storing the data. (**Fun fact:** On 6 November 1998, version 1.0.0 of the HDF5 library and file format was released.)
6. **Community and Ecosystem:** The HDF5community and ecosystem are notable for their diversity, collaboration, and continuous growth, reflecting the wide range of applications and industries that utilize the HDF5 framework. HDF5 is used across various sectors including academic research, government agencies, high-performance computing, aerospace, finance, and environmental science. This diversity leads to a rich community with a wide array of use cases and expertise. Users and developers actively engage through mailing lists, forums, and social media platforms. The HDF Forum is particularly notable for its active discussions and exchange of knowledge. The HDF Group, which oversees the development of HDF5, often collaborates with the user community in identifying priorities, improving features, and fixing bugs. This collaborative approach ensures that HDF5 continues to meet the evolving needs of its users. The HDF5 community is global, with users and contributors from around the world. This international aspect brings diverse perspectives and use cases to the community. There is a wealth of educational resources available, including tutorials, webinars, workshops, and documentation, which help new users learn about HDF5 and assist experienced users in tackling more complex tasks. Regular conferences, workshops, and webinars are held, often organized by the HDF Group or in conjunction with scientific conferences, fostering community interaction and knowledge sharing. Being open source, HDF5 benefits from contributions from individuals and organizations worldwide. This includes code contributions, bug reports, and feature requests. The HDF Group often partners with academic institutions, research organizations, and industry leaders on various projects, ensuring that HDF5 continues to align with cutting-edge data needs.